In [1]:
import pandas as pd
from utils import data_util as du, preproc_utils as pu, extractor_util as exu, estimator_util as esu
import importlib
from typing import List, Dict, Any
import wrappers as wrp
import os

In [2]:
DATA_SRC_DIR = '/Users/yoav/Documents/Yoav/CS/22928 - Intro to CV/cv-final/data'
EXT_DATA_DIR = '/Users/yoav/Documents/Yoav/CS/22928 - Intro to CV/cv-final/data/external'
INT_DATA_DIR = '/Users/yoav/Documents/Yoav/CS/22928 - Intro to CV/cv-final/data/internal'

In [54]:
importlib.reload(wrp)
importlib.reload(du)
importlib.reload(exu)
importlib.reload(esu)

<module 'utils.estimator_util' from '/Users/yoav/Documents/Yoav/CS/22928 - Intro to CV/cv-final/utils/estimator_util.py'>

In [55]:
# Usage example
dataset = du.DatasetLoader(EXT_DATA_DIR)

In [56]:
# Usage
config = pu.PreprocessConfig(size=(512, 512), normalize=False)
preprocessor = pu.ImagePreprocessor(config)

In [57]:
default_sift_config = exu.SIFTConfig()
extractor_config = exu.FeatureExtractorConfig(default_sift_config)
extractor = exu.FeatureExtractor(config=extractor_config)

In [58]:
dataset = wrp.load_dataset(EXT_DATA_DIR)

In [59]:
dataset = wrp.preprocess_data(dataset, preprocessor, extractor)

In [64]:
dataset = wrp.extract_features(dataset, extractor)

In [65]:
bf_config = exu.BFMatcherConfig(knn=True)
matcher_config = exu.MatcherConfig(bf_config)

In [66]:
matcher = exu.FeatureMatcher(matcher_config)

In [127]:
kpt = dataset.scenes_data['brandenburg_gate'].image_data['02035158_8518636650'].features.keypoints

In [128]:
kpt = [p.pt for p in kpt]

In [129]:
kt = dataset.scenes_data['brandenburg_gate'].calibration.loc['02035158_8518636650'].camera_intrinsics

In [84]:
import utils.general_utils as gu

In [130]:
importlib.reload(gu)

<module 'utils.general_utils' from '/Users/yoav/Documents/Yoav/CS/22928 - Intro to CV/cv-final/utils/general_utils.py'>

In [131]:
gu.normalize_keypoints(kpt, kt)

array([[-0.21514718, -0.19124969],
       [-0.21511119, -0.19877724],
       [-0.21511119, -0.19877724],
       ...,
       [ 0.06869601, -0.10120926],
       [ 0.06916943, -0.1512334 ],
       [ 0.06918809, -0.06553081]], shape=(1876, 2))

In [72]:
kt[0,0]

np.float64(1777.77002)

In [154]:
wrp.match_features(dataset, matcher, covisibility_threshold=0.8)

Matching features for scene: brandenburg_gate
There are 129 valid pairs to estimate for
[[-0.44961144 -0.05436471]
 [-0.44086675 -0.18174262]
 [-0.43915076 -0.18092303]
 [-0.43915076 -0.18092303]]
[[-0.44961144 -0.05436471]
 [-0.4456796  -0.18422198]
 [-0.44462959 -0.17812532]
 [-0.43074827 -0.16571657]]
[[-0.4703503  -0.04322744]
 [-0.46614415 -0.05546648]
 [-0.46199499 -0.1892807 ]
 [-0.46199499 -0.1892807 ]]
[[-0.47223691 -0.06281873]
 [-0.45874993 -0.18993584]
 [-0.457428   -0.1921713 ]
 [-0.45684495 -0.18419226]]
[[-0.5398442  -0.20045234]
 [-0.53900853 -0.19982869]
 [-0.53817761 -0.18714259]
 [-0.53663118 -0.1869381 ]]
[[-0.15105542  0.01099782]
 [-0.15077293  0.02775741]
 [-0.14981815 -0.02242241]
 [-0.14852839 -0.02284974]]
[[-0.15078268  0.00805073]
 [-0.13655282  0.0348357 ]
 [-0.13599298  0.01575451]
 [-0.13024358 -0.00153497]]
[[-0.15077293  0.02775741]
 [-0.1479768  -0.012008  ]
 [-0.14731479  0.01959674]
 [-0.13599298  0.01575451]]
[[-0.14112816  0.00693369]
 [-0.13610003

In [155]:
dataset.scenes_data['brandenburg_gate'].covisibility

,pair,covisibility,fundamental_matrix,im1,im2,keypoints1,keypoints2
0,90920828_5082887495-20133057_3035445116,0.905,-9.27158516e-04 1.22034601e-01 -9.87725452e+01...,90920828_5082887495,20133057_3035445116,"(-0.449611, -0.054365);(-0.440867, -0.181743);...","(-0.466144, -0.055466);(-0.455768, -0.185925);..."
1,90920828_5082887495-17262282_1141017004,0.936,1.68899800e-03 1.18269247e-01 -1.02939518e+02 ...,90920828_5082887495,17262282_1141017004,"(-0.449611, -0.054365);(-0.445680, -0.184222);...","(-0.461805, -0.083045);(-0.021437, -0.182804);..."
2,20133057_3035445116-17262282_1141017004,0.908,-5.56308440e-04 1.74170978e-02 -2.49391042e+01...,20133057_3035445116,17262282_1141017004,"(-0.470350, -0.043227);(-0.466144, -0.055466);...","(-0.467542, -0.070519);(-0.461805, -0.083045);..."
3,38600512_2168650655-17262282_1141017004,0.901,1.26837683e-02 -5.42924979e-01 4.29820975e+02 ...,38600512_2168650655,17262282_1141017004,"(-0.472237, -0.062819);(-0.458750, -0.189936);...","(-0.206424, -0.179398);(-0.021437, -0.182804);..."
4,60770994_853214983-17262282_1141017004,0.912,-1.05214974e-02 -4.87393290e-01 3.90468870e+02...,60770994_853214983,17262282_1141017004,"(-0.539844, -0.200452);(-0.539009, -0.199829);...","(-0.021437, -0.182804);(-0.021437, -0.182804);..."
...,...,...,...,...,...,...,...
7498,66720928_3418295149-65128211_1900926684,0.076,-8.12702062e-03 2.55588179e-01 -1.69505309e+02...,66720928_3418295149,65128211_1900926684,NaN,NaN
7499,67266149_8114963267-20580890_6015148266,0.055,3.67415708e-03 6.55116510e-01 -4.49956197e+02 ...,67266149_8114963267,20580890_6015148266,NaN,NaN
7500,67266149_8114963267-52630012_12435805525,0.094,-9.33957978e-02 2.05087664e+00 -1.31472168e+03...,67266149_8114963267,52630012_12435805525,NaN,NaN
7501,67266149_8114963267-63229395_2704362565,0.042,-1.92191439e-02 8.12448695e-01 -5.57773493e+02...,67266149_8114963267,63229395_2704362565,NaN,NaN


In [138]:
es_alg_conf = esu.RANSACConfig()
es_conf = esu.EstimatorConfig('RANSAC', es_alg_conf)

In [139]:
list1 = [(1, 2), (3, 4), (5, 6), (7, 8), (9, 10), (11, 12), (13, 14), (15, 16)]
list2 = [(17, 18), (19, 20), (21, 22), (23, 24), (25, 26), (27, 28), (29, 30), (31, 32)]

In [140]:
fe = esu.FundamentalMatrixEstimator(list1, list2, es_conf)

In [156]:
sub = wrp.estimate_fundamental_matrix(dataset=dataset, estimator=fe)

Estimating fundamental matrix for pairs in scene: brandenburg_gate


In [157]:
sub

,sample_id,fundamental_matrix,mask,inliers1,inliers2
0,brandenburg_gate;90920828_5082887495-20133057_...,"[12.727305249369623, 129.8952782434918, 33.798...","[[1], [1], [1], [1], [1], [1], [1], [1], [1], ...","[[-0.449611, -0.054365], [-0.440867, -0.181743...","[[-0.466144, -0.055466], [-0.455768, -0.185925..."
1,brandenburg_gate;90920828_5082887495-17262282_...,"[0.8136154091033959, 1.4789251634568656, 1.624...","[[1], [1], [1], [1], [1], [1], [1], [1], [1], ...","[[-0.449611, -0.054365], [-0.44568, -0.184222]...","[[-0.461805, -0.083045], [-0.021437, -0.182804..."
2,brandenburg_gate;20133057_3035445116-17262282_...,"[8.050046364629091, 30.007080418407753, 8.7334...","[[1], [1], [1], [1], [1], [1], [1], [1], [1], ...","[[-0.47035, -0.043227], [-0.466144, -0.055466]...","[[-0.467542, -0.070519], [-0.461805, -0.083045..."
3,brandenburg_gate;38600512_2168650655-17262282_...,"[1.9424634411315012, -11.466641154098864, -0.9...","[[1], [1], [1], [1], [1], [1], [1], [1], [1], ...","[[-0.472237, -0.062819], [-0.45875, -0.189936]...","[[-0.206424, -0.179398], [-0.021437, -0.182804..."
4,brandenburg_gate;60770994_853214983-17262282_1...,"[-12.213286566785511, -55.69894294595479, -16....","[[1], [1], [1], [1], [1], [1], [1], [1], [1], ...","[[-0.539844, -0.200452], [-0.539009, -0.199829...","[[-0.021437, -0.182804], [-0.021437, -0.182804..."
...,...,...,...,...,...
124,brandenburg_gate;60824110_1469539304-08468928_...,"[-8.213053501459155, 146.14638226789347, 38.72...","[[1], [1], [1], [1], [1], [1], [1], [1], [1], ...","[[-0.399234, 0.129181], [-0.393517, -0.116796]...","[[-0.241999, -0.202796], [-0.379758, -0.080159..."
125,brandenburg_gate;66720928_3418295149-20133057_...,"[20.700259524451795, -558.3868505686797, -136....","[[1], [1], [1], [1], [1], [1], [1], [1], [1], ...","[[-0.451147, -0.031407], [-0.43672, -0.079455]...","[[-0.044686, -0.09846], [-0.466144, -0.055466]..."
126,brandenburg_gate;66720928_3418295149-02936509_...,"[2.1700889810077104, 390.277154039344, -15.016...","[[1], [1], [1], [1], [1], [1], [1], [1], [1], ...","[[-0.44896, -0.088448], [-0.43465, 0.031499], ...","[[-0.422117, -0.320985], [-0.258219, -0.349716..."
127,brandenburg_gate;66720928_3418295149-08814095_...,"[43.12498319973662, -238.8020090557619, -49.64...","[[1], [1], [1], [1], [1], [1], [1], [1], [1], ...","[[-0.455657, -0.079186], [-0.444088, -0.04479]...","[[-0.480328, -0.073369], [-0.463585, -0.038242..."


In [167]:
demo = sub.loc[0]
demo

sample_id             brandenburg_gate;90920828_5082887495-20133057_...
fundamental_matrix    [12.727305249369623, 129.8952782434918, 33.798...
mask                  [[1], [1], [1], [1], [1], [1], [1], [1], [1], ...
inliers1              [[-0.449611, -0.054365], [-0.440867, -0.181743...
inliers2              [[-0.466144, -0.055466], [-0.455768, -0.185925...
Name: 0, dtype: object

In [179]:
demo.fundamental_matrix.reshape(3,3)

array([[  12.72730525,  129.89527824,   33.79829022],
       [-127.73674212,   -1.13817873,  -33.95838174],
       [ -27.73220476,   35.47896317,    1.        ]])

In [186]:
dataset.scenes_data['brandenburg_gate'].covisibility.set_index('pair', inplace=True)

In [190]:
import cv2 as cv

In [237]:
importlib.reload(du)
importlib.reload(exu)
importlib.reload(esu)
importlib.reload(gu)

/Users/yoav/Documents/Yoav/CS/22928 - Intro to CV/cv-final
<module 'utils' (namespace) from ['/Users/yoav/Documents/Yoav/CS/22928 - Intro to CV/cv-final/utils']>
data_util
estimator_util
evaluation_util
extractor_util
general_utils
preproc_utils


ModuleNotFoundError: No module named 'general_utils'

In [233]:
import os
print(os.getcwd())

/Users/yoav/Documents/Yoav/CS/22928 - Intro to CV/cv-final


In [242]:
importlib.reload(wrp)


<module 'wrappers' from '/Users/yoav/Documents/Yoav/CS/22928 - Intro to CV/cv-final/wrappers.py'>

In [ ]:
wrp.evaluate_results(dataset, sub)